<a href="https://colab.research.google.com/github/GiovanniBorrelli/Software-Requirements-to-User-Stories/blob/main/LLAMA_already_drive_mounted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Mount Google Drive (only if not already mounted)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Set environment variables for the Ollama model and package directories
import os
os.environ['OLLAMA_MODEL_DIR'] = '/content/drive/MyDrive/ollama_models'
os.environ['OLLAMA_PACKAGE_DIR'] = '/content/drive/MyDrive/colab_packages'

# Check if pciutils and Ollama API are already installed
!dpkg -l | grep pciutils || sudo apt-get install -y pciutils
!if [ ! -f /usr/local/bin/ollama ]; then curl -fsSL https://ollama.com/install.sh | sh; fi

# Create a Python script to start the Ollama API server in a separate thread
import threading
import subprocess

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

# Start the Ollama API server in a separate thread
ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

# Only download the model if it's not already in Google Drive
model_path = os.path.join(os.environ['OLLAMA_MODEL_DIR'], 'llama3.1_8b')
if not os.path.exists(model_path):
    !ollama pull llama3.1:8b

# Install lightrag only if it's not already in Google Drive
import sys
sys.path.append(os.environ['OLLAMA_PACKAGE_DIR'])

try:
    import lightrag
except ImportError:
    !pip install -U lightrag[ollama] --target=$OLLAMA_PACKAGE_DIR

from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient

# Template for QA system
qa_template = r"""<SYS>
You are a helpful assistant.
</SYS>
User: {{input_str}}
You:"""

# Define the SimpleQA class
class SimpleQA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input: dict) -> str:
        return self.generator.call({"input_str": str(input)})

    async def acall(self, input: dict) -> str:
        return await self.generator.acall({"input_str": str(input)})

from lightrag.components.model_client import OllamaClient
from IPython.display import Markdown, display
# Initialize the model client
model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "llama3.1:8b"}
}
qa = SimpleQA(**model)


Mounted at /content/drive
ii  pciutils                               1:3.7.0-6                               amd64        PCI utilities
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕▏   96 B                         
pulling 1a4c3c319823... 100% ▕▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [13]:
output=qa("""Make USER STORIES from the software requirements below.  these requirements designed to meet specific standards or guidelines in healthcare informatics, such as those from the HITSP (Health IT Standards Panel), IHE (Integrating the Healthcare Enterprise), and align with HIPAA (Health Insurance Portability and Accountability Act) regulations for protecting patient health information.
MAKE A LIST OF ALL USER STORIES! DONT JUST TELL ME THE KEY AREAS. DO IT, EVEN IF THE USER STORIES ARE MORE THAN 50!
Requirements:The system shall create and maintain a single patient record for each patient, storing key identifier information (e.g., system ID, medical record number) with the ability to store multiple identifiers.
The system shall provide the ability to merge duplicate patient records into a single patient record.
The system shall capture and maintain demographic information, including historic data (e.g., prior names, addresses), with the ability to modify and access this information as discrete data elements for patient care and reporting functions.
The system shall allow querying for a patient using multiple forms of identification.
The system shall capture and maintain the identity of all providers involved in a patient's care, including the principal provider.
The system shall maintain a coded list of problems or diagnoses, capturing details such as onset date, resolution date, chronicity, and any associated orders or medications, with the ability to update, display, and document changes.
The system shall capture and display free text comments associated with problems or diagnoses.
The system shall maintain and display a medication list for each patient, including prescribing details, medication history, and dates (ordering, start, modify, renewal, and end). The system shall support uncoded or free text medications and document changes.
The system shall provide the ability to manage a patient-specific medication list, including current medications, with functionality to exclude or print medications and document that a patient takes no medications.
The system shall capture and maintain an allergy and adverse reaction list, including user modifications, the type of reaction, and an indication of whether the list was reviewed. It shall also indicate if the patient has no known drug allergies or adverse reactions.
The system shall store, display, and manage comprehensive patient history, including medication and allergy information as discrete data elements.
The system shall capture, update, and display patient history, including structured, coded, and free text data, with the ability to record the presence and absence of conditions and capture history from external sources.
The system shall provide the ability to manage clinical documentation, including saving notes in progress, finalizing, cosigning, addending, and correcting notes. The system shall track and display the identity of users making changes and the date/time of those changes.
The system shall allow filtering, searching, and ordering of notes by provider or associated diagnosis, with the ability to enter free text notes and customize templates for structured data input.
The system shall capture and manage patient vital signs and other clinical data elements as discrete data, displaying measurements in both metric and English units, and indicating when values fall outside preset normal ranges. The system shall graph and calculate metrics like BMI over time.

 """)
display(Markdown(f"**Answer:** {output.data}"))

**Answer:** Here is the list of user stories:

1. **Create a single patient record**: As a healthcare provider, I want to create a single patient record for each patient, so that all their key identifier information is stored in one place.
2. **Store multiple identifiers per patient**: As a healthcare provider, I want to store multiple identifiers (e.g., system ID, medical record number) for each patient, so that I can easily track and identify them.
3. **Merge duplicate patient records**: As a healthcare administrator, I want to merge duplicate patient records into a single patient record, so that patient information is accurate and up-to-date.
4. **Capture demographic information**: As a healthcare provider, I want to capture and maintain demographic information (e.g., prior names, addresses) for each patient, so that I can better understand their needs and provide personalized care.
5. **Modify and access historical data**: As a healthcare provider, I want to modify and access historical demographic information as discrete data elements, so that I can see how patients' needs have changed over time.
6. **Query patients by multiple identifiers**: As a healthcare provider, I want to query for patients using multiple forms of identification (e.g., name, date of birth, medical record number), so that I can easily locate them in the system.
7. **Capture principal provider information**: As a healthcare administrator, I want to capture and maintain the identity of all providers involved in a patient's care, including the principal provider, so that we can track their roles and responsibilities.
8. **Maintain coded list of problems or diagnoses**: As a healthcare provider, I want to maintain a coded list of problems or diagnoses for each patient, capturing details such as onset date, resolution date, chronicity, and associated orders or medications.
9. **Update, display, and document changes to coded list**: As a healthcare provider, I want to update, display, and document changes to the coded list of problems or diagnoses, so that patients' conditions are accurately reflected in their records.
10. **Capture free text comments associated with problems or diagnoses**: As a healthcare provider, I want to capture and maintain free text comments associated with problems or diagnoses, so that additional context can be provided about patients' conditions.
11. **Maintain medication list for each patient**: As a healthcare provider, I want to maintain and display a medication list for each patient, including prescribing details, medication history, and dates (ordering, start, modify, renewal, and end).
12. **Support uncoded or free text medications**: As a healthcare provider, I want the system to support uncoded or free text medications and document changes, so that we can accurately capture patients' medication regimens.
13. **Manage patient-specific medication list**: As a healthcare provider, I want to manage a patient-specific medication list, including current medications, with functionality to exclude or print medications and document that a patient takes no medications.
14. **Capture allergy and adverse reaction information**: As a healthcare provider, I want to capture and maintain an allergy and adverse reaction list for each patient, including user modifications, the type of reaction, and an indication of whether the list was reviewed.
15. **Document absence of known drug allergies or adverse reactions**: As a healthcare provider, I want the system to document if a patient has no known drug allergies or adverse reactions, so that we can ensure their safety when prescribing medications.
16. **Store comprehensive patient history**: As a healthcare provider, I want to store and display comprehensive patient history, including medication and allergy information as discrete data elements.
17. **Capture, update, and display patient history**: As a healthcare provider, I want to capture, update, and display patient history, including structured, coded, and free text data, with the ability to record the presence and absence of conditions and capture history from external sources.
18. **Record conditions as present or absent**: As a healthcare provider, I want to be able to record conditions as present or absent in patients' records, so that we can accurately track their health status over time.
19. **Capture history from external sources**: As a healthcare provider, I want the system to allow capture of patient history from external sources (e.g., other EHR systems), so that we can ensure complete and accurate information is available for care decisions.
20. **Manage clinical documentation**: As a healthcare provider, I want to manage clinical documentation, including saving notes in progress, finalizing, cosigning, addending, and correcting notes, as well as tracking changes made by users.
21. **Track changes made to clinical documentation**: As a healthcare administrator, I want the system to track and display the identity of users making changes to clinical documentation and the date/time of those changes, so that we can maintain audit trails for accountability and compliance.
22. **Filter, search, and order notes by provider or diagnosis**: As a healthcare provider, I want to filter, search, and order notes by provider or associated diagnosis, with the ability to enter free text notes and customize templates for structured data input.
23. **Capture patient vital signs and clinical data elements**: As a healthcare provider, I want to capture and manage patient vital signs and other clinical data elements as discrete data, displaying measurements in both metric and English units.
24. **Graph and calculate metrics like BMI over time**: As a healthcare provider, I want the system to graph and calculate metrics like BMI over time, so that we can track patients' health status and make informed care decisions.

Note: These user stories are designed to meet specific standards or guidelines in healthcare informatics, such as those from HITSP, IHE, and HIPAA regulations for protecting patient health information.

In [ ]:
## IGNORE THIS CODE

!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
from IPython.display import clear_output

# Create a Python script to start the Ollama API server in a separate thread

import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

from IPython.display import clear_output
!ollama pull llama3.1:8b
clear_output()

!pip install -U lightrag[ollama]

from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient, GroqAPIClient

import time

qa_template = r"""<SYS>
You are a helpful assistant.
</SYS>
User: {{input_str}}
You:"""

class SimpleQA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input: dict) -> str:
        return self.generator.call({"input_str": str(input)})

    async def acall(self, input: dict) -> str:
        return await self.generator.acall({"input_str": str(input)})

from lightrag.components.model_client import OllamaClient
from IPython.display import Markdown, display
model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "llama3.1:8b"}
}
qa = SimpleQA(**model)